## Homework Six:  Compiling a C-like Languages:  Loops and Conditionals

## General Homework Policies for CS 320

Please provide answers to these problems by filling in this notebook and submitting it via websubmit by <span style="color:red">Tuesday3/27 by midnight. You may submit it by Wednesday 3/28 for a 10% penalty.</span> If the lab for this week involved a separate notebook, it is due at the same time. For any problems involving diagrams or drawings, you may try your hand at ASCII art or you may submit an electronic version of your drawings, either using drawing software or by scanning or photographing hand drawings. A message has been posted to Piazza showing you how to automatically include these in your notebook (a nice touch).  Make sure in the latter case that the result is legible; if we can not understand your drawing, it will receive no credit. 

There is a grace period, longer than 15 minutes but shorter than a couple of hours, after the deadlines, to account for temporarily problems with submission (dropped connections etc.). In cases where I have uploaded
a solution late on Wednesday night (probably close to the 15 minute mark!) no submissions will be accepted after the solution is uploaded. 

I will drop the lowest homework/lab combination at the end of the term.

For coding problems any <i> reasonable </i> solution, in which you understand the spirit of the exercise and try to learn this week's material through your efforts, will be graded based solely on the test cases. In general, shortcuts to avoid learning the material will be unreasonable. For example, you may not use any Python libraries or functions that make your task dramatically easier (e.g., you may use split(), replace(...), int(...), and float(...), as mentioned below, and any other normal functions, but may NOT use the Python library for regular expressions).  Good Python style, using list comprehensions, Numpy, creating helper functions in addition to what is required, etc., <i>when appropriate,</i> are quite reasonable! Check with me if you have questions about this. 
<p>
<span style="color:red"><b>Lab problems</b> may be worked on collaboratively at the lab and afterwards, with no need to indicate whom you collaborated with. (Of course, if you simply copy, then you will not learn what you need for homeworks and tests, and your grade will reflect your lack of effort.)</span> 

<span style="color:red"><b>Homework problems</b> must be solved individually, and will be subject to the College rules on plagiarism. <b>In particular, you are absolutely forbidden to copy code or answers from another student or from a source other than my slides and materials I provide to you.</b> We will run the code from notebooks through an electronic service that checks for plagiarism, and any violation will be dealt with harshly, up to and including an automatic drop of one letter grade at the end of the term (this policy will be familiar to those who took CS 111 at BU). Significant offences will be referred to the College Academic Misconduct Committee. These policies are necessary to provide a fair environment for you to do your best work with the expectation that the quality and quantity of your efforts will be rewarded appropriately. <b>Please take this as seriously as I do.</b></span> 

<b> Make sure to click Cell -> Run All before submitting notebooks!!</b>

### Lexical analyzer code. DO NOT MODIFY!

We have added a couple of new tokens for manipulating lists: nil, cons, first, and rest. 

In [128]:

# Token Categories -- slightly different than in HW01 and HW 02

ident = 'id'          # used in token as (indent, <string> )    
integer = 'int'       #  (integer, <value> )
floating = 'float'    #  (floating, <value> )  
trueTok = 'True'
falseTok = 'False'
noneTok = "None"
plus = '+'            #  (plus,)    rest as this one, no attribute necessary
minus = '-'          
mult = '*'
div = '/' 
mod = '%'
exp = '**'
lparen = '('
rparen = ')'
lbrace = '['
rbrace = ']'
equals = '=='
assign = '='
colon = ':'
lt = '<'
gt = '>'
ge = '>='
le = '<='
ne = '!='
andTok = 'and'
orTok = 'or'
notTok = 'not'
andTok2 = '&&'
orTok2 = '||'
notTok2 = '!'
defTok = 'def'
semicolon = ';'
comma = ','
let = 'let'
inTok = 'in'
ifTok = 'if'
thenTok = 'then'
elseTok = 'else'
lambdaTok = 'lambda'
apply = 'apply'
returnTok = 'return'
nil = 'nil'
cons = 'cons'
first = 'first'
rest = 'rest'
whileTok = 'while'
leftCurlyBrace = '{'
rightCurlyBrace = '}'
returnTok = 'return'
printTok = 'print'
error = 'error'       #  (error, <string> )     gives spelling of token that caused the lexical error

token_list = [trueTok,
falseTok,
noneTok,
plus,
minus,       
mult,
div,
mod,
exp ,
lparen ,
rparen ,
lbrace ,
rbrace ,
equals ,
assign ,
colon ,
lt ,
gt ,
ge ,
le ,
ne ,
andTok ,
orTok ,
notTok ,
andTok2 ,
orTok2 ,
notTok2 ,
defTok ,
semicolon ,
comma ,
let ,
inTok ,
ifTok ,
thenTok ,
elseTok ,
lambdaTok ,
apply ,
returnTok ,
nil ,
cons ,
first ,
rest ,
whileTok ,
leftCurlyBrace ,
rightCurlyBrace ,
returnTok ,
printTok ,
error  ]




# special token for end of input

end_of_input = '$'       # (end_of_input,) will be pretty-printed as ($,)

"""
def tokenToString(t):
    if t == None:
        return str(t)
    elif t[0] in ['int','float','id']:
        return '(' + t[0] + ',' + str(t[1]) + ')'
    else:
        return '(' + t[0] + ',)'
"""
    
def tokenToString(t):
    if t == None:
        return str(t)
    elif t[0] in ['int','float','id']:
        return str(t[1])
    else:
        return t[0] 
        
def tokenListToString(lst):
    res = '[ '
    for t in lst[:-1]:
        res = res + tokenToString(t) + ', '
    res = res + tokenToString(lst[-1]) + ' ]'
    return res


# Code for lexer

# put white space between each separator or operator token and split into words

def splitTokens(s):
    for t in ['+','-','*','/','(',')','[',']','{', '}',';',',','<', ':', '>', '=', '!','%']:
        s = s.replace(t,' ' + t + ' ')
    # now repair two-character tokens
    s = s.replace('<  =','<=')
    s = s.replace('>  =','>=')
    s = s.replace('=  =','==')
    s = s.replace('!  =','!=')
    s = s.replace('*  *','**')
    s = s.replace('&  &','&&')
    s = s.replace('|  |','||')
    return s.split()

def isLetter(c):
    return 'a' <= c <= 'z' or 'A' <= c <= 'Z'

def isDigit(c):
    return '0' <= c <= '9' 

def isIdToken(s):
    state = 1
    for ch in s:
        if state == 1:
            if isLetter(ch) or ch == '_':
                state = 2
            else:
                return False
        elif state == 2:
            if isLetter(ch) or isDigit(ch) or ch == '_':
                state = 2
            else:
                return False
    return (state == 2)

                        
def isIntToken(s):
    if s == '0':
        return True
    state = 1
    for ch in s:
        if state == 1:
            if isDigit(ch) and ch != '0':
                state = 2
            else:
                return False
        elif state == 2:
            if isDigit(ch):
                state = 2
            else:
                return False
    return (state == 2)
    
def isFloatToken(s):
    state = 1
    finalStates = [4,6]
    for ch in s:
        if state == 1:
            if isDigit(ch) and ch != '0':
                state = 2
            elif ch == '0':
                state = 3
            elif ch == '.':
                state = 5
            else:
                return False
        elif state == 2:
            if isDigit(ch):
                state = 2
            elif ch == '.':
                state = 4
            else:
                return False
        elif state == 3:
            if ch == '.':
                state = 4
            else:
                return False
        elif state == 4:
            if isDigit(ch):
                state = 4
            else:
                return False
        elif state == 5:
            if isDigit(ch):
                state = 6
            else:
                return False
        elif state == 6:
            if isDigit(ch):
                state = 6
            else:
                return False                
                
    return (state in finalStates)   
    


def stringToToken(t):
    if t in token_list:
        return (t,)
    elif isIdToken(t):
        return (ident,t)
    elif isIntToken(t):
        return (integer,int(t)) 
    elif isFloatToken(t):
        return (floating,float(t)) 

    else:
        return (error,)
    

def lexer(s):
    return [stringToToken(t) for t in splitTokens(s)]


In [129]:
# Utilities for grammars

def rule_to_string(r):
    s = r[0][0] + ' := '
    for t in r[1]:
        s = s + t[0] + ' '
    return s
                          
def pprint_grammar(G): 
    for k in range(len(G)):
        print(str(k) + ": " + rule_to_string(G[k])) 
  
Grammar_HW06 =    [(('S',),(('Stmts',),)),          # program is a list of statements
         (('Stmts',),(('Stmt',),(';',), ('Stmts',))),    
         (('Stmts',),(('Stmt',),(';',))),
         (('Stmts',),(('Block',), ('Stmts',))),    
         (('Stmts',),(('Block',),)), 
         (('Stmt',),(('id',),('=',),('Expr',))), 
         (('Stmt',),(('print' ,),('Expr',))),
         (('Block',),(('{',),('Stmts',),('}',))), 
         (('Block',),(('While',),('WhileExpr',),('Block',))),
         (('WhileExpr',),(('(',),('Expr',),(')',))),
         (('While',),(('while',),)),
         (('Block',),(('IfHeader',),('Block',))),
         (('IfHeader',),(('if',),('(',),('Expr',),(')',))),
         (('Block',),(('IfHeader',),('Block',),('Else',),('Block',))), 
         (('Else',),(('else',),)), 
         (('Expr',),(('Bor',),)),  
         (('Bor',),(('Bor',),('||',),('Band',))),   
         (('Bor',),(('Band',),)),                   
         (('Band',),(('Band',),('&&',),('Bnot',))),
         (('Band',),(('Bnot',),)),                 
         (('Bnot',),(('!',),('Bnot',))),          
         (('Bnot',),(('C',),)),                     
         (('C',),(('E',),('==',),('E',))),          
         (('C',),(('E',),('!=',),('E',))),          
         (('C',),(('E',),('<',),('E',))),            
         (('C',),(('E',),('<=',),('E',))),          
         (('C',),(('E',),('>',),('E',))),            
         (('C',),(('E',),('>=',),('E',))),          
         (('C',),(('E',))),                         
         (('E',),(('E',),('+',),('T',))),           
         (('E',),(('E',),('-',),('T',))),           
         (('E',),(('T',),)),                       
         (('T',),(('T',),('*',),('F',))),          
         (('T',),(('T',),('/',),('F',))),          
         (('T',),(('T',),('%',),('F',))),          
         (('T',),(('F',),)),                       
         (('F',),(('-',),('F',))),                                                                                                      
         (('F',),(('id',),)),                      
         (('F',),(('int',),)),                    
         (('F',),(('float',),)),                    
         (('F',),(('(',),('Expr',),(')',))) ]     

pprint_grammar(Grammar_HW06)

0: S := Stmts 
1: Stmts := Stmt ; Stmts 
2: Stmts := Stmt ; 
3: Stmts := Block Stmts 
4: Stmts := Block 
5: Stmt := id = Expr 
6: Stmt := print Expr 
7: Block := { Stmts } 
8: Block := While WhileExpr Block 
9: WhileExpr := ( Expr ) 
10: While := while 
11: Block := IfHeader Block 
12: IfHeader := if ( Expr ) 
13: Block := IfHeader Block Else Block 
14: Else := else 
15: Expr := Bor 
16: Bor := Bor || Band 
17: Bor := Band 
18: Band := Band && Bnot 
19: Band := Bnot 
20: Bnot := ! Bnot 
21: Bnot := C 
22: C := E == E 
23: C := E != E 
24: C := E < E 
25: C := E <= E 
26: C := E > E 
27: C := E >= E 
28: C := E 
29: E := E + T 
30: E := E - T 
31: E := T 
32: T := T * F 
33: T := T / F 
34: T := T % F 
35: T := F 
36: F := - F 
37: F := id 
38: F := int 
39: F := float 
40: F := ( Expr ) 


#   Parser 

### The next two cells contain provided code for the homework; DO NOT MODIFY!

Changes from the last time are the table for the new grammar, in which the columns are indexed by strings and now err = None and Accept = 0. You should
not have to know this, but just in case....


In [130]:
# DFA to drive the shift-reduce parser

# The parser DFA will tell you what to do when the stack is
# in a given configuration.  A positive number gives the
# state transition, a non-positive number k is a reduction by the
# rule -k, i.e., -4 means a reduction by rule 4 with no
# advance in the input string.  None is an error and
# 0 is accept. 

err = None
accept = 0

table = {
  0:{'{': 4, 'else': None, '+': None, 'WhileExpr': None, 'While': 2, 'id': 3, 'print': 5, '!=': None, 'F': None, 'while': 6, '%': None, '==': None, ';': None, '>': None, 'Expr': None, 'Band': None, 'if': 10, '||': None, 'Else': None, '<': None, 'int': None, '<=': None, '}': None, 'T': None, 'Stmt': 9, ')': None, '(': None, '=': None, '/': None, '>=': None, 'S': None, 'IfHeader': 1, 'E': None, 'C': None, 'float': None, 'Bnot': None, '$': None, 'Bor': None, 'Stmts': 7, '!': None, 'Block': 8, '*': None, '&&': None, '-': None},
  1:{'{': 4, 'else': None, '+': None, 'WhileExpr': None, 'While': 2, 'id': None, 'print': None, '!=': None, 'F': None, 'while': 6, '%': None, '==': None, ';': None, '>': None, 'Expr': None, 'Band': None, 'if': 10, '||': None, 'Else': None, '<': None, 'int': None, '<=': None, '}': None, 'T': None, 'Stmt': None, ')': None, '(': None, '=': None, '/': None, '>=': None, 'S': None, 'IfHeader': 1, 'E': None, 'C': None, 'float': None, 'Bnot': None, '$': None, 'Bor': None, 'Stmts': None, '!': None, 'Block': 11, '*': None, '&&': None, '-': None},
  2:{'{': None, 'else': None, '+': None, 'WhileExpr': 12, 'While': None, 'id': None, 'print': None, '!=': None, 'F': None, 'while': None, '%': None, '==': None, ';': None, '>': None, 'Expr': None, 'Band': None, 'if': None, '||': None, 'Else': None, '<': None, 'int': None, '<=': None, '}': None, 'T': None, 'Stmt': None, ')': None, '(': 13, '=': None, '/': None, '>=': None, 'S': None, 'IfHeader': None, 'E': None, 'C': None, 'float': None, 'Bnot': None, '$': None, 'Bor': None, 'Stmts': None, '!': None, 'Block': None, '*': None, '&&': None, '-': None},
  3:{'{': None, 'else': None, '+': None, 'WhileExpr': None, 'While': None, 'id': None, 'print': None, '!=': None, 'F': None, 'while': None, '%': None, '==': None, ';': None, '>': None, 'Expr': None, 'Band': None, 'if': None, '||': None, 'Else': None, '<': None, 'int': None, '<=': None, '}': None, 'T': None, 'Stmt': None, ')': None, '(': None, '=': 14, '/': None, '>=': None, 'S': None, 'IfHeader': None, 'E': None, 'C': None, 'float': None, 'Bnot': None, '$': None, 'Bor': None, 'Stmts': None, '!': None, 'Block': None, '*': None, '&&': None, '-': None},
  4:{'{': 4, 'else': None, '+': None, 'WhileExpr': None, 'While': 2, 'id': 3, 'print': 5, '!=': None, 'F': None, 'while': 6, '%': None, '==': None, ';': None, '>': None, 'Expr': None, 'Band': None, 'if': 10, '||': None, 'Else': None, '<': None, 'int': None, '<=': None, '}': None, 'T': None, 'Stmt': 9, ')': None, '(': None, '=': None, '/': None, '>=': None, 'S': None, 'IfHeader': 1, 'E': None, 'C': None, 'float': None, 'Bnot': None, '$': None, 'Bor': None, 'Stmts': 15, '!': None, 'Block': 8, '*': None, '&&': None, '-': None},
  5:{'{': None, 'else': None, '+': None, 'WhileExpr': None, 'While': None, 'id': 18, 'print': None, '!=': None, 'F': 19, 'while': None, '%': None, '==': None, ';': None, '>': None, 'Expr': 26, 'Band': 27, 'if': None, '||': None, 'Else': None, '<': None, 'int': 23, '<=': None, '}': None, 'T': 25, 'Stmt': None, ')': None, '(': 29, '=': None, '/': None, '>=': None, 'S': None, 'IfHeader': None, 'E': 16, 'C': 28, 'float': 17, 'Bnot': 20, '$': None, 'Bor': 21, 'Stmts': None, '!': 24, 'Block': None, '*': None, '&&': None, '-': 22},
  6:{'{': None, 'else': None, '+': None, 'WhileExpr': None, 'While': None, 'id': None, 'print': None, '!=': None, 'F': None, 'while': None, '%': None, '==': None, ';': None, '>': None, 'Expr': None, 'Band': None, 'if': None, '||': None, 'Else': None, '<': None, 'int': None, '<=': None, '}': None, 'T': None, 'Stmt': None, ')': None, '(': -10, '=': None, '/': None, '>=': None, 'S': None, 'IfHeader': None, 'E': None, 'C': None, 'float': None, 'Bnot': None, '$': None, 'Bor': None, 'Stmts': None, '!': None, 'Block': None, '*': None, '&&': None, '-': None},
  7:{'{': None, 'else': None, '+': None, 'WhileExpr': None, 'While': None, 'id': None, 'print': None, '!=': None, 'F': None, 'while': None, '%': None, '==': None, ';': None, '>': None, 'Expr': None, 'Band': None, 'if': None, '||': None, 'Else': None, '<': None, 'int': None, '<=': None, '}': None, 'T': None, 'Stmt': None, ')': None, '(': None, '=': None, '/': None, '>=': None, 'S': None, 'IfHeader': None, 'E': None, 'C': None, 'float': None, 'Bnot': None, '$': 0, 'Bor': None, 'Stmts': None, '!': None, 'Block': None, '*': None, '&&': None, '-': None},
  8:{'{': 4, 'else': None, '+': None, 'WhileExpr': None, 'While': 2, 'id': 3, 'print': 5, '!=': None, 'F': None, 'while': 6, '%': None, '==': None, ';': None, '>': None, 'Expr': None, 'Band': None, 'if': 10, '||': None, 'Else': None, '<': None, 'int': None, '<=': None, '}': -4, 'T': None, 'Stmt': 9, ')': None, '(': None, '=': None, '/': None, '>=': None, 'S': None, 'IfHeader': 1, 'E': None, 'C': None, 'float': None, 'Bnot': None, '$': -4, 'Bor': None, 'Stmts': 30, '!': None, 'Block': 8, '*': None, '&&': None, '-': None},
  9:{'{': None, 'else': None, '+': None, 'WhileExpr': None, 'While': None, 'id': None, 'print': None, '!=': None, 'F': None, 'while': None, '%': None, '==': None, ';': 31, '>': None, 'Expr': None, 'Band': None, 'if': None, '||': None, 'Else': None, '<': None, 'int': None, '<=': None, '}': None, 'T': None, 'Stmt': None, ')': None, '(': None, '=': None, '/': None, '>=': None, 'S': None, 'IfHeader': None, 'E': None, 'C': None, 'float': None, 'Bnot': None, '$': None, 'Bor': None, 'Stmts': None, '!': None, 'Block': None, '*': None, '&&': None, '-': None},
  10:{'{': None, 'else': None, '+': None, 'WhileExpr': None, 'While': None, 'id': None, 'print': None, '!=': None, 'F': None, 'while': None, '%': None, '==': None, ';': None, '>': None, 'Expr': None, 'Band': None, 'if': None, '||': None, 'Else': None, '<': None, 'int': None, '<=': None, '}': None, 'T': None, 'Stmt': None, ')': None, '(': 32, '=': None, '/': None, '>=': None, 'S': None, 'IfHeader': None, 'E': None, 'C': None, 'float': None, 'Bnot': None, '$': None, 'Bor': None, 'Stmts': None, '!': None, 'Block': None, '*': None, '&&': None, '-': None},
  11:{'{': -11, 'else': 33, '+': None, 'WhileExpr': None, 'While': None, 'id': -11, 'print': -11, '!=': None, 'F': None, 'while': -11, '%': None, '==': None, ';': None, '>': None, 'Expr': None, 'Band': None, 'if': -11, '||': None, 'Else': 34, '<': None, 'int': None, '<=': None, '}': -11, 'T': None, 'Stmt': None, ')': None, '(': None, '=': None, '/': None, '>=': None, 'S': None, 'IfHeader': None, 'E': None, 'C': None, 'float': None, 'Bnot': None, '$': -11, 'Bor': None, 'Stmts': None, '!': None, 'Block': None, '*': None, '&&': None, '-': None},
  12:{'{': 4, 'else': None, '+': None, 'WhileExpr': None, 'While': 2, 'id': None, 'print': None, '!=': None, 'F': None, 'while': 6, '%': None, '==': None, ';': None, '>': None, 'Expr': None, 'Band': None, 'if': 10, '||': None, 'Else': None, '<': None, 'int': None, '<=': None, '}': None, 'T': None, 'Stmt': None, ')': None, '(': None, '=': None, '/': None, '>=': None, 'S': None, 'IfHeader': 1, 'E': None, 'C': None, 'float': None, 'Bnot': None, '$': None, 'Bor': None, 'Stmts': None, '!': None, 'Block': 35, '*': None, '&&': None, '-': None},
  13:{'{': None, 'else': None, '+': None, 'WhileExpr': None, 'While': None, 'id': 18, 'print': None, '!=': None, 'F': 19, 'while': None, '%': None, '==': None, ';': None, '>': None, 'Expr': 36, 'Band': 27, 'if': None, '||': None, 'Else': None, '<': None, 'int': 23, '<=': None, '}': None, 'T': 25, 'Stmt': None, ')': None, '(': 29, '=': None, '/': None, '>=': None, 'S': None, 'IfHeader': None, 'E': 16, 'C': 28, 'float': 17, 'Bnot': 20, '$': None, 'Bor': 21, 'Stmts': None, '!': 24, 'Block': None, '*': None, '&&': None, '-': 22},
  14:{'{': None, 'else': None, '+': None, 'WhileExpr': None, 'While': None, 'id': 18, 'print': None, '!=': None, 'F': 19, 'while': None, '%': None, '==': None, ';': None, '>': None, 'Expr': 37, 'Band': 27, 'if': None, '||': None, 'Else': None, '<': None, 'int': 23, '<=': None, '}': None, 'T': 25, 'Stmt': None, ')': None, '(': 29, '=': None, '/': None, '>=': None, 'S': None, 'IfHeader': None, 'E': 16, 'C': 28, 'float': 17, 'Bnot': 20, '$': None, 'Bor': 21, 'Stmts': None, '!': 24, 'Block': None, '*': None, '&&': None, '-': 22},
  15:{'{': None, 'else': None, '+': None, 'WhileExpr': None, 'While': None, 'id': None, 'print': None, '!=': None, 'F': None, 'while': None, '%': None, '==': None, ';': None, '>': None, 'Expr': None, 'Band': None, 'if': None, '||': None, 'Else': None, '<': None, 'int': None, '<=': None, '}': 38, 'T': None, 'Stmt': None, ')': None, '(': None, '=': None, '/': None, '>=': None, 'S': None, 'IfHeader': None, 'E': None, 'C': None, 'float': None, 'Bnot': None, '$': None, 'Bor': None, 'Stmts': None, '!': None, 'Block': None, '*': None, '&&': None, '-': None},
  16:{'{': None, 'else': None, '+': 45, 'WhileExpr': None, 'While': None, 'id': None, 'print': None, '!=': 41, 'F': None, 'while': None, '%': None, '==': 42, ';': -28, '>': 43, 'Expr': None, 'Band': None, 'if': None, '||': -28, 'Else': None, '<': 39, 'int': None, '<=': 44, '}': None, 'T': None, 'Stmt': None, ')': -28, '(': None, '=': None, '/': None, '>=': 40, 'S': None, 'IfHeader': None, 'E': None, 'C': None, 'float': None, 'Bnot': None, '$': None, 'Bor': None, 'Stmts': None, '!': None, 'Block': None, '*': None, '&&': -28, '-': 46},
  17:{'{': None, 'else': None, '+': -39, 'WhileExpr': None, 'While': None, 'id': None, 'print': None, '!=': -39, 'F': None, 'while': None, '%': -39, '==': -39, ';': -39, '>': -39, 'Expr': None, 'Band': None, 'if': None, '||': -39, 'Else': None, '<': -39, 'int': None, '<=': -39, '}': None, 'T': None, 'Stmt': None, ')': -39, '(': None, '=': None, '/': -39, '>=': -39, 'S': None, 'IfHeader': None, 'E': None, 'C': None, 'float': None, 'Bnot': None, '$': None, 'Bor': None, 'Stmts': None, '!': None, 'Block': None, '*': -39, '&&': -39, '-': -39},
  18:{'{': None, 'else': None, '+': -37, 'WhileExpr': None, 'While': None, 'id': None, 'print': None, '!=': -37, 'F': None, 'while': None, '%': -37, '==': -37, ';': -37, '>': -37, 'Expr': None, 'Band': None, 'if': None, '||': -37, 'Else': None, '<': -37, 'int': None, '<=': -37, '}': None, 'T': None, 'Stmt': None, ')': -37, '(': None, '=': None, '/': -37, '>=': -37, 'S': None, 'IfHeader': None, 'E': None, 'C': None, 'float': None, 'Bnot': None, '$': None, 'Bor': None, 'Stmts': None, '!': None, 'Block': None, '*': -37, '&&': -37, '-': -37},
  19:{'{': None, 'else': None, '+': -35, 'WhileExpr': None, 'While': None, 'id': None, 'print': None, '!=': -35, 'F': None, 'while': None, '%': -35, '==': -35, ';': -35, '>': -35, 'Expr': None, 'Band': None, 'if': None, '||': -35, 'Else': None, '<': -35, 'int': None, '<=': -35, '}': None, 'T': None, 'Stmt': None, ')': -35, '(': None, '=': None, '/': -35, '>=': -35, 'S': None, 'IfHeader': None, 'E': None, 'C': None, 'float': None, 'Bnot': None, '$': None, 'Bor': None, 'Stmts': None, '!': None, 'Block': None, '*': -35, '&&': -35, '-': -35},
  20:{'{': None, 'else': None, '+': None, 'WhileExpr': None, 'While': None, 'id': None, 'print': None, '!=': None, 'F': None, 'while': None, '%': None, '==': None, ';': -19, '>': None, 'Expr': None, 'Band': None, 'if': None, '||': -19, 'Else': None, '<': None, 'int': None, '<=': None, '}': None, 'T': None, 'Stmt': None, ')': -19, '(': None, '=': None, '/': None, '>=': None, 'S': None, 'IfHeader': None, 'E': None, 'C': None, 'float': None, 'Bnot': None, '$': None, 'Bor': None, 'Stmts': None, '!': None, 'Block': None, '*': None, '&&': -19, '-': None},
  21:{'{': None, 'else': None, '+': None, 'WhileExpr': None, 'While': None, 'id': None, 'print': None, '!=': None, 'F': None, 'while': None, '%': None, '==': None, ';': -15, '>': None, 'Expr': None, 'Band': None, 'if': None, '||': 47, 'Else': None, '<': None, 'int': None, '<=': None, '}': None, 'T': None, 'Stmt': None, ')': -15, '(': None, '=': None, '/': None, '>=': None, 'S': None, 'IfHeader': None, 'E': None, 'C': None, 'float': None, 'Bnot': None, '$': None, 'Bor': None, 'Stmts': None, '!': None, 'Block': None, '*': None, '&&': None, '-': None},
  22:{'{': None, 'else': None, '+': None, 'WhileExpr': None, 'While': None, 'id': 18, 'print': None, '!=': None, 'F': 48, 'while': None, '%': None, '==': None, ';': None, '>': None, 'Expr': None, 'Band': None, 'if': None, '||': None, 'Else': None, '<': None, 'int': 23, '<=': None, '}': None, 'T': None, 'Stmt': None, ')': None, '(': 29, '=': None, '/': None, '>=': None, 'S': None, 'IfHeader': None, 'E': None, 'C': None, 'float': 17, 'Bnot': None, '$': None, 'Bor': None, 'Stmts': None, '!': None, 'Block': None, '*': None, '&&': None, '-': 22},
  23:{'{': None, 'else': None, '+': -38, 'WhileExpr': None, 'While': None, 'id': None, 'print': None, '!=': -38, 'F': None, 'while': None, '%': -38, '==': -38, ';': -38, '>': -38, 'Expr': None, 'Band': None, 'if': None, '||': -38, 'Else': None, '<': -38, 'int': None, '<=': -38, '}': None, 'T': None, 'Stmt': None, ')': -38, '(': None, '=': None, '/': -38, '>=': -38, 'S': None, 'IfHeader': None, 'E': None, 'C': None, 'float': None, 'Bnot': None, '$': None, 'Bor': None, 'Stmts': None, '!': None, 'Block': None, '*': -38, '&&': -38, '-': -38},
  24:{'{': None, 'else': None, '+': None, 'WhileExpr': None, 'While': None, 'id': 18, 'print': None, '!=': None, 'F': 19, 'while': None, '%': None, '==': None, ';': None, '>': None, 'Expr': None, 'Band': None, 'if': None, '||': None, 'Else': None, '<': None, 'int': 23, '<=': None, '}': None, 'T': 25, 'Stmt': None, ')': None, '(': 29, '=': None, '/': None, '>=': None, 'S': None, 'IfHeader': None, 'E': 16, 'C': 28, 'float': 17, 'Bnot': 49, '$': None, 'Bor': None, 'Stmts': None, '!': 24, 'Block': None, '*': None, '&&': None, '-': 22},
  25:{'{': None, 'else': None, '+': -31, 'WhileExpr': None, 'While': None, 'id': None, 'print': None, '!=': -31, 'F': None, 'while': None, '%': 51, '==': -31, ';': -31, '>': -31, 'Expr': None, 'Band': None, 'if': None, '||': -31, 'Else': None, '<': -31, 'int': None, '<=': -31, '}': None, 'T': None, 'Stmt': None, ')': -31, '(': None, '=': None, '/': 52, '>=': -31, 'S': None, 'IfHeader': None, 'E': None, 'C': None, 'float': None, 'Bnot': None, '$': None, 'Bor': None, 'Stmts': None, '!': None, 'Block': None, '*': 50, '&&': -31, '-': -31},
  26:{'{': None, 'else': None, '+': None, 'WhileExpr': None, 'While': None, 'id': None, 'print': None, '!=': None, 'F': None, 'while': None, '%': None, '==': None, ';': -6, '>': None, 'Expr': None, 'Band': None, 'if': None, '||': None, 'Else': None, '<': None, 'int': None, '<=': None, '}': None, 'T': None, 'Stmt': None, ')': None, '(': None, '=': None, '/': None, '>=': None, 'S': None, 'IfHeader': None, 'E': None, 'C': None, 'float': None, 'Bnot': None, '$': None, 'Bor': None, 'Stmts': None, '!': None, 'Block': None, '*': None, '&&': None, '-': None},
  27:{'{': None, 'else': None, '+': None, 'WhileExpr': None, 'While': None, 'id': None, 'print': None, '!=': None, 'F': None, 'while': None, '%': None, '==': None, ';': -17, '>': None, 'Expr': None, 'Band': None, 'if': None, '||': -17, 'Else': None, '<': None, 'int': None, '<=': None, '}': None, 'T': None, 'Stmt': None, ')': -17, '(': None, '=': None, '/': None, '>=': None, 'S': None, 'IfHeader': None, 'E': None, 'C': None, 'float': None, 'Bnot': None, '$': None, 'Bor': None, 'Stmts': None, '!': None, 'Block': None, '*': None, '&&': 53, '-': None},
  28:{'{': None, 'else': None, '+': None, 'WhileExpr': None, 'While': None, 'id': None, 'print': None, '!=': None, 'F': None, 'while': None, '%': None, '==': None, ';': -21, '>': None, 'Expr': None, 'Band': None, 'if': None, '||': -21, 'Else': None, '<': None, 'int': None, '<=': None, '}': None, 'T': None, 'Stmt': None, ')': -21, '(': None, '=': None, '/': None, '>=': None, 'S': None, 'IfHeader': None, 'E': None, 'C': None, 'float': None, 'Bnot': None, '$': None, 'Bor': None, 'Stmts': None, '!': None, 'Block': None, '*': None, '&&': -21, '-': None},
  29:{'{': None, 'else': None, '+': None, 'WhileExpr': None, 'While': None, 'id': 18, 'print': None, '!=': None, 'F': 19, 'while': None, '%': None, '==': None, ';': None, '>': None, 'Expr': 54, 'Band': 27, 'if': None, '||': None, 'Else': None, '<': None, 'int': 23, '<=': None, '}': None, 'T': 25, 'Stmt': None, ')': None, '(': 29, '=': None, '/': None, '>=': None, 'S': None, 'IfHeader': None, 'E': 16, 'C': 28, 'float': 17, 'Bnot': 20, '$': None, 'Bor': 21, 'Stmts': None, '!': 24, 'Block': None, '*': None, '&&': None, '-': 22},
  30:{'{': None, 'else': None, '+': None, 'WhileExpr': None, 'While': None, 'id': None, 'print': None, '!=': None, 'F': None, 'while': None, '%': None, '==': None, ';': None, '>': None, 'Expr': None, 'Band': None, 'if': None, '||': None, 'Else': None, '<': None, 'int': None, '<=': None, '}': -3, 'T': None, 'Stmt': None, ')': None, '(': None, '=': None, '/': None, '>=': None, 'S': None, 'IfHeader': None, 'E': None, 'C': None, 'float': None, 'Bnot': None, '$': -3, 'Bor': None, 'Stmts': None, '!': None, 'Block': None, '*': None, '&&': None, '-': None},
  31:{'{': 4, 'else': None, '+': None, 'WhileExpr': None, 'While': 2, 'id': 3, 'print': 5, '!=': None, 'F': None, 'while': 6, '%': None, '==': None, ';': None, '>': None, 'Expr': None, 'Band': None, 'if': 10, '||': None, 'Else': None, '<': None, 'int': None, '<=': None, '}': -2, 'T': None, 'Stmt': 9, ')': None, '(': None, '=': None, '/': None, '>=': None, 'S': None, 'IfHeader': 1, 'E': None, 'C': None, 'float': None, 'Bnot': None, '$': -2, 'Bor': None, 'Stmts': 55, '!': None, 'Block': 8, '*': None, '&&': None, '-': None},
  32:{'{': None, 'else': None, '+': None, 'WhileExpr': None, 'While': None, 'id': 18, 'print': None, '!=': None, 'F': 19, 'while': None, '%': None, '==': None, ';': None, '>': None, 'Expr': 56, 'Band': 27, 'if': None, '||': None, 'Else': None, '<': None, 'int': 23, '<=': None, '}': None, 'T': 25, 'Stmt': None, ')': None, '(': 29, '=': None, '/': None, '>=': None, 'S': None, 'IfHeader': None, 'E': 16, 'C': 28, 'float': 17, 'Bnot': 20, '$': None, 'Bor': 21, 'Stmts': None, '!': 24, 'Block': None, '*': None, '&&': None, '-': 22},
  33:{'{': -14, 'else': None, '+': None, 'WhileExpr': None, 'While': None, 'id': None, 'print': None, '!=': None, 'F': None, 'while': -14, '%': None, '==': None, ';': None, '>': None, 'Expr': None, 'Band': None, 'if': -14, '||': None, 'Else': None, '<': None, 'int': None, '<=': None, '}': None, 'T': None, 'Stmt': None, ')': None, '(': None, '=': None, '/': None, '>=': None, 'S': None, 'IfHeader': None, 'E': None, 'C': None, 'float': None, 'Bnot': None, '$': None, 'Bor': None, 'Stmts': None, '!': None, 'Block': None, '*': None, '&&': None, '-': None},
  34:{'{': 4, 'else': None, '+': None, 'WhileExpr': None, 'While': 2, 'id': None, 'print': None, '!=': None, 'F': None, 'while': 6, '%': None, '==': None, ';': None, '>': None, 'Expr': None, 'Band': None, 'if': 10, '||': None, 'Else': None, '<': None, 'int': None, '<=': None, '}': None, 'T': None, 'Stmt': None, ')': None, '(': None, '=': None, '/': None, '>=': None, 'S': None, 'IfHeader': 1, 'E': None, 'C': None, 'float': None, 'Bnot': None, '$': None, 'Bor': None, 'Stmts': None, '!': None, 'Block': 57, '*': None, '&&': None, '-': None},
  35:{'{': -8, 'else': -8, '+': None, 'WhileExpr': None, 'While': None, 'id': -8, 'print': -8, '!=': None, 'F': None, 'while': -8, '%': None, '==': None, ';': None, '>': None, 'Expr': None, 'Band': None, 'if': -8, '||': None, 'Else': None, '<': None, 'int': None, '<=': None, '}': -8, 'T': None, 'Stmt': None, ')': None, '(': None, '=': None, '/': None, '>=': None, 'S': None, 'IfHeader': None, 'E': None, 'C': None, 'float': None, 'Bnot': None, '$': -8, 'Bor': None, 'Stmts': None, '!': None, 'Block': None, '*': None, '&&': None, '-': None},
  36:{'{': None, 'else': None, '+': None, 'WhileExpr': None, 'While': None, 'id': None, 'print': None, '!=': None, 'F': None, 'while': None, '%': None, '==': None, ';': None, '>': None, 'Expr': None, 'Band': None, 'if': None, '||': None, 'Else': None, '<': None, 'int': None, '<=': None, '}': None, 'T': None, 'Stmt': None, ')': 58, '(': None, '=': None, '/': None, '>=': None, 'S': None, 'IfHeader': None, 'E': None, 'C': None, 'float': None, 'Bnot': None, '$': None, 'Bor': None, 'Stmts': None, '!': None, 'Block': None, '*': None, '&&': None, '-': None},
  37:{'{': None, 'else': None, '+': None, 'WhileExpr': None, 'While': None, 'id': None, 'print': None, '!=': None, 'F': None, 'while': None, '%': None, '==': None, ';': -5, '>': None, 'Expr': None, 'Band': None, 'if': None, '||': None, 'Else': None, '<': None, 'int': None, '<=': None, '}': None, 'T': None, 'Stmt': None, ')': None, '(': None, '=': None, '/': None, '>=': None, 'S': None, 'IfHeader': None, 'E': None, 'C': None, 'float': None, 'Bnot': None, '$': None, 'Bor': None, 'Stmts': None, '!': None, 'Block': None, '*': None, '&&': None, '-': None},
  38:{'{': -7, 'else': -7, '+': None, 'WhileExpr': None, 'While': None, 'id': -7, 'print': -7, '!=': None, 'F': None, 'while': -7, '%': None, '==': None, ';': None, '>': None, 'Expr': None, 'Band': None, 'if': -7, '||': None, 'Else': None, '<': None, 'int': None, '<=': None, '}': -7, 'T': None, 'Stmt': None, ')': None, '(': None, '=': None, '/': None, '>=': None, 'S': None, 'IfHeader': None, 'E': None, 'C': None, 'float': None, 'Bnot': None, '$': -7, 'Bor': None, 'Stmts': None, '!': None, 'Block': None, '*': None, '&&': None, '-': None},
  39:{'{': None, 'else': None, '+': None, 'WhileExpr': None, 'While': None, 'id': 18, 'print': None, '!=': None, 'F': 19, 'while': None, '%': None, '==': None, ';': None, '>': None, 'Expr': None, 'Band': None, 'if': None, '||': None, 'Else': None, '<': None, 'int': 23, '<=': None, '}': None, 'T': 25, 'Stmt': None, ')': None, '(': 29, '=': None, '/': None, '>=': None, 'S': None, 'IfHeader': None, 'E': 59, 'C': None, 'float': 17, 'Bnot': None, '$': None, 'Bor': None, 'Stmts': None, '!': None, 'Block': None, '*': None, '&&': None, '-': 22},
  40:{'{': None, 'else': None, '+': None, 'WhileExpr': None, 'While': None, 'id': 18, 'print': None, '!=': None, 'F': 19, 'while': None, '%': None, '==': None, ';': None, '>': None, 'Expr': None, 'Band': None, 'if': None, '||': None, 'Else': None, '<': None, 'int': 23, '<=': None, '}': None, 'T': 25, 'Stmt': None, ')': None, '(': 29, '=': None, '/': None, '>=': None, 'S': None, 'IfHeader': None, 'E': 60, 'C': None, 'float': 17, 'Bnot': None, '$': None, 'Bor': None, 'Stmts': None, '!': None, 'Block': None, '*': None, '&&': None, '-': 22},
  41:{'{': None, 'else': None, '+': None, 'WhileExpr': None, 'While': None, 'id': 18, 'print': None, '!=': None, 'F': 19, 'while': None, '%': None, '==': None, ';': None, '>': None, 'Expr': None, 'Band': None, 'if': None, '||': None, 'Else': None, '<': None, 'int': 23, '<=': None, '}': None, 'T': 25, 'Stmt': None, ')': None, '(': 29, '=': None, '/': None, '>=': None, 'S': None, 'IfHeader': None, 'E': 61, 'C': None, 'float': 17, 'Bnot': None, '$': None, 'Bor': None, 'Stmts': None, '!': None, 'Block': None, '*': None, '&&': None, '-': 22},
  42:{'{': None, 'else': None, '+': None, 'WhileExpr': None, 'While': None, 'id': 18, 'print': None, '!=': None, 'F': 19, 'while': None, '%': None, '==': None, ';': None, '>': None, 'Expr': None, 'Band': None, 'if': None, '||': None, 'Else': None, '<': None, 'int': 23, '<=': None, '}': None, 'T': 25, 'Stmt': None, ')': None, '(': 29, '=': None, '/': None, '>=': None, 'S': None, 'IfHeader': None, 'E': 62, 'C': None, 'float': 17, 'Bnot': None, '$': None, 'Bor': None, 'Stmts': None, '!': None, 'Block': None, '*': None, '&&': None, '-': 22},
  43:{'{': None, 'else': None, '+': None, 'WhileExpr': None, 'While': None, 'id': 18, 'print': None, '!=': None, 'F': 19, 'while': None, '%': None, '==': None, ';': None, '>': None, 'Expr': None, 'Band': None, 'if': None, '||': None, 'Else': None, '<': None, 'int': 23, '<=': None, '}': None, 'T': 25, 'Stmt': None, ')': None, '(': 29, '=': None, '/': None, '>=': None, 'S': None, 'IfHeader': None, 'E': 63, 'C': None, 'float': 17, 'Bnot': None, '$': None, 'Bor': None, 'Stmts': None, '!': None, 'Block': None, '*': None, '&&': None, '-': 22},
  44:{'{': None, 'else': None, '+': None, 'WhileExpr': None, 'While': None, 'id': 18, 'print': None, '!=': None, 'F': 19, 'while': None, '%': None, '==': None, ';': None, '>': None, 'Expr': None, 'Band': None, 'if': None, '||': None, 'Else': None, '<': None, 'int': 23, '<=': None, '}': None, 'T': 25, 'Stmt': None, ')': None, '(': 29, '=': None, '/': None, '>=': None, 'S': None, 'IfHeader': None, 'E': 64, 'C': None, 'float': 17, 'Bnot': None, '$': None, 'Bor': None, 'Stmts': None, '!': None, 'Block': None, '*': None, '&&': None, '-': 22},
  45:{'{': None, 'else': None, '+': None, 'WhileExpr': None, 'While': None, 'id': 18, 'print': None, '!=': None, 'F': 19, 'while': None, '%': None, '==': None, ';': None, '>': None, 'Expr': None, 'Band': None, 'if': None, '||': None, 'Else': None, '<': None, 'int': 23, '<=': None, '}': None, 'T': 65, 'Stmt': None, ')': None, '(': 29, '=': None, '/': None, '>=': None, 'S': None, 'IfHeader': None, 'E': None, 'C': None, 'float': 17, 'Bnot': None, '$': None, 'Bor': None, 'Stmts': None, '!': None, 'Block': None, '*': None, '&&': None, '-': 22},
  46:{'{': None, 'else': None, '+': None, 'WhileExpr': None, 'While': None, 'id': 18, 'print': None, '!=': None, 'F': 19, 'while': None, '%': None, '==': None, ';': None, '>': None, 'Expr': None, 'Band': None, 'if': None, '||': None, 'Else': None, '<': None, 'int': 23, '<=': None, '}': None, 'T': 66, 'Stmt': None, ')': None, '(': 29, '=': None, '/': None, '>=': None, 'S': None, 'IfHeader': None, 'E': None, 'C': None, 'float': 17, 'Bnot': None, '$': None, 'Bor': None, 'Stmts': None, '!': None, 'Block': None, '*': None, '&&': None, '-': 22},
  47:{'{': None, 'else': None, '+': None, 'WhileExpr': None, 'While': None, 'id': 18, 'print': None, '!=': None, 'F': 19, 'while': None, '%': None, '==': None, ';': None, '>': None, 'Expr': None, 'Band': 67, 'if': None, '||': None, 'Else': None, '<': None, 'int': 23, '<=': None, '}': None, 'T': 25, 'Stmt': None, ')': None, '(': 29, '=': None, '/': None, '>=': None, 'S': None, 'IfHeader': None, 'E': 16, 'C': 28, 'float': 17, 'Bnot': 20, '$': None, 'Bor': None, 'Stmts': None, '!': 24, 'Block': None, '*': None, '&&': None, '-': 22},
  48:{'{': None, 'else': None, '+': -36, 'WhileExpr': None, 'While': None, 'id': None, 'print': None, '!=': -36, 'F': None, 'while': None, '%': -36, '==': -36, ';': -36, '>': -36, 'Expr': None, 'Band': None, 'if': None, '||': -36, 'Else': None, '<': -36, 'int': None, '<=': -36, '}': None, 'T': None, 'Stmt': None, ')': -36, '(': None, '=': None, '/': -36, '>=': -36, 'S': None, 'IfHeader': None, 'E': None, 'C': None, 'float': None, 'Bnot': None, '$': None, 'Bor': None, 'Stmts': None, '!': None, 'Block': None, '*': -36, '&&': -36, '-': -36},
  49:{'{': None, 'else': None, '+': None, 'WhileExpr': None, 'While': None, 'id': None, 'print': None, '!=': None, 'F': None, 'while': None, '%': None, '==': None, ';': -20, '>': None, 'Expr': None, 'Band': None, 'if': None, '||': -20, 'Else': None, '<': None, 'int': None, '<=': None, '}': None, 'T': None, 'Stmt': None, ')': -20, '(': None, '=': None, '/': None, '>=': None, 'S': None, 'IfHeader': None, 'E': None, 'C': None, 'float': None, 'Bnot': None, '$': None, 'Bor': None, 'Stmts': None, '!': None, 'Block': None, '*': None, '&&': -20, '-': None},
  50:{'{': None, 'else': None, '+': None, 'WhileExpr': None, 'While': None, 'id': 18, 'print': None, '!=': None, 'F': 68, 'while': None, '%': None, '==': None, ';': None, '>': None, 'Expr': None, 'Band': None, 'if': None, '||': None, 'Else': None, '<': None, 'int': 23, '<=': None, '}': None, 'T': None, 'Stmt': None, ')': None, '(': 29, '=': None, '/': None, '>=': None, 'S': None, 'IfHeader': None, 'E': None, 'C': None, 'float': 17, 'Bnot': None, '$': None, 'Bor': None, 'Stmts': None, '!': None, 'Block': None, '*': None, '&&': None, '-': 22},
  51:{'{': None, 'else': None, '+': None, 'WhileExpr': None, 'While': None, 'id': 18, 'print': None, '!=': None, 'F': 69, 'while': None, '%': None, '==': None, ';': None, '>': None, 'Expr': None, 'Band': None, 'if': None, '||': None, 'Else': None, '<': None, 'int': 23, '<=': None, '}': None, 'T': None, 'Stmt': None, ')': None, '(': 29, '=': None, '/': None, '>=': None, 'S': None, 'IfHeader': None, 'E': None, 'C': None, 'float': 17, 'Bnot': None, '$': None, 'Bor': None, 'Stmts': None, '!': None, 'Block': None, '*': None, '&&': None, '-': 22},
  52:{'{': None, 'else': None, '+': None, 'WhileExpr': None, 'While': None, 'id': 18, 'print': None, '!=': None, 'F': 70, 'while': None, '%': None, '==': None, ';': None, '>': None, 'Expr': None, 'Band': None, 'if': None, '||': None, 'Else': None, '<': None, 'int': 23, '<=': None, '}': None, 'T': None, 'Stmt': None, ')': None, '(': 29, '=': None, '/': None, '>=': None, 'S': None, 'IfHeader': None, 'E': None, 'C': None, 'float': 17, 'Bnot': None, '$': None, 'Bor': None, 'Stmts': None, '!': None, 'Block': None, '*': None, '&&': None, '-': 22},
  53:{'{': None, 'else': None, '+': None, 'WhileExpr': None, 'While': None, 'id': 18, 'print': None, '!=': None, 'F': 19, 'while': None, '%': None, '==': None, ';': None, '>': None, 'Expr': None, 'Band': None, 'if': None, '||': None, 'Else': None, '<': None, 'int': 23, '<=': None, '}': None, 'T': 25, 'Stmt': None, ')': None, '(': 29, '=': None, '/': None, '>=': None, 'S': None, 'IfHeader': None, 'E': 16, 'C': 28, 'float': 17, 'Bnot': 71, '$': None, 'Bor': None, 'Stmts': None, '!': 24, 'Block': None, '*': None, '&&': None, '-': 22},
  54:{'{': None, 'else': None, '+': None, 'WhileExpr': None, 'While': None, 'id': None, 'print': None, '!=': None, 'F': None, 'while': None, '%': None, '==': None, ';': None, '>': None, 'Expr': None, 'Band': None, 'if': None, '||': None, 'Else': None, '<': None, 'int': None, '<=': None, '}': None, 'T': None, 'Stmt': None, ')': 72, '(': None, '=': None, '/': None, '>=': None, 'S': None, 'IfHeader': None, 'E': None, 'C': None, 'float': None, 'Bnot': None, '$': None, 'Bor': None, 'Stmts': None, '!': None, 'Block': None, '*': None, '&&': None, '-': None},
  55:{'{': None, 'else': None, '+': None, 'WhileExpr': None, 'While': None, 'id': None, 'print': None, '!=': None, 'F': None, 'while': None, '%': None, '==': None, ';': None, '>': None, 'Expr': None, 'Band': None, 'if': None, '||': None, 'Else': None, '<': None, 'int': None, '<=': None, '}': -1, 'T': None, 'Stmt': None, ')': None, '(': None, '=': None, '/': None, '>=': None, 'S': None, 'IfHeader': None, 'E': None, 'C': None, 'float': None, 'Bnot': None, '$': -1, 'Bor': None, 'Stmts': None, '!': None, 'Block': None, '*': None, '&&': None, '-': None},
  56:{'{': None, 'else': None, '+': None, 'WhileExpr': None, 'While': None, 'id': None, 'print': None, '!=': None, 'F': None, 'while': None, '%': None, '==': None, ';': None, '>': None, 'Expr': None, 'Band': None, 'if': None, '||': None, 'Else': None, '<': None, 'int': None, '<=': None, '}': None, 'T': None, 'Stmt': None, ')': 73, '(': None, '=': None, '/': None, '>=': None, 'S': None, 'IfHeader': None, 'E': None, 'C': None, 'float': None, 'Bnot': None, '$': None, 'Bor': None, 'Stmts': None, '!': None, 'Block': None, '*': None, '&&': None, '-': None},
  57:{'{': -13, 'else': -13, '+': None, 'WhileExpr': None, 'While': None, 'id': -13, 'print': -13, '!=': None, 'F': None, 'while': -13, '%': None, '==': None, ';': None, '>': None, 'Expr': None, 'Band': None, 'if': -13, '||': None, 'Else': None, '<': None, 'int': None, '<=': None, '}': -13, 'T': None, 'Stmt': None, ')': None, '(': None, '=': None, '/': None, '>=': None, 'S': None, 'IfHeader': None, 'E': None, 'C': None, 'float': None, 'Bnot': None, '$': -13, 'Bor': None, 'Stmts': None, '!': None, 'Block': None, '*': None, '&&': None, '-': None},
  58:{'{': -9, 'else': None, '+': None, 'WhileExpr': None, 'While': None, 'id': None, 'print': None, '!=': None, 'F': None, 'while': -9, '%': None, '==': None, ';': None, '>': None, 'Expr': None, 'Band': None, 'if': -9, '||': None, 'Else': None, '<': None, 'int': None, '<=': None, '}': None, 'T': None, 'Stmt': None, ')': None, '(': None, '=': None, '/': None, '>=': None, 'S': None, 'IfHeader': None, 'E': None, 'C': None, 'float': None, 'Bnot': None, '$': None, 'Bor': None, 'Stmts': None, '!': None, 'Block': None, '*': None, '&&': None, '-': None},
  59:{'{': None, 'else': None, '+': 45, 'WhileExpr': None, 'While': None, 'id': None, 'print': None, '!=': None, 'F': None, 'while': None, '%': None, '==': None, ';': -24, '>': None, 'Expr': None, 'Band': None, 'if': None, '||': -24, 'Else': None, '<': None, 'int': None, '<=': None, '}': None, 'T': None, 'Stmt': None, ')': -24, '(': None, '=': None, '/': None, '>=': None, 'S': None, 'IfHeader': None, 'E': None, 'C': None, 'float': None, 'Bnot': None, '$': None, 'Bor': None, 'Stmts': None, '!': None, 'Block': None, '*': None, '&&': -24, '-': 46},
  60:{'{': None, 'else': None, '+': 45, 'WhileExpr': None, 'While': None, 'id': None, 'print': None, '!=': None, 'F': None, 'while': None, '%': None, '==': None, ';': -27, '>': None, 'Expr': None, 'Band': None, 'if': None, '||': -27, 'Else': None, '<': None, 'int': None, '<=': None, '}': None, 'T': None, 'Stmt': None, ')': -27, '(': None, '=': None, '/': None, '>=': None, 'S': None, 'IfHeader': None, 'E': None, 'C': None, 'float': None, 'Bnot': None, '$': None, 'Bor': None, 'Stmts': None, '!': None, 'Block': None, '*': None, '&&': -27, '-': 46},
  61:{'{': None, 'else': None, '+': 45, 'WhileExpr': None, 'While': None, 'id': None, 'print': None, '!=': None, 'F': None, 'while': None, '%': None, '==': None, ';': -23, '>': None, 'Expr': None, 'Band': None, 'if': None, '||': -23, 'Else': None, '<': None, 'int': None, '<=': None, '}': None, 'T': None, 'Stmt': None, ')': -23, '(': None, '=': None, '/': None, '>=': None, 'S': None, 'IfHeader': None, 'E': None, 'C': None, 'float': None, 'Bnot': None, '$': None, 'Bor': None, 'Stmts': None, '!': None, 'Block': None, '*': None, '&&': -23, '-': 46},
  62:{'{': None, 'else': None, '+': 45, 'WhileExpr': None, 'While': None, 'id': None, 'print': None, '!=': None, 'F': None, 'while': None, '%': None, '==': None, ';': -22, '>': None, 'Expr': None, 'Band': None, 'if': None, '||': -22, 'Else': None, '<': None, 'int': None, '<=': None, '}': None, 'T': None, 'Stmt': None, ')': -22, '(': None, '=': None, '/': None, '>=': None, 'S': None, 'IfHeader': None, 'E': None, 'C': None, 'float': None, 'Bnot': None, '$': None, 'Bor': None, 'Stmts': None, '!': None, 'Block': None, '*': None, '&&': -22, '-': 46},
  63:{'{': None, 'else': None, '+': 45, 'WhileExpr': None, 'While': None, 'id': None, 'print': None, '!=': None, 'F': None, 'while': None, '%': None, '==': None, ';': -26, '>': None, 'Expr': None, 'Band': None, 'if': None, '||': -26, 'Else': None, '<': None, 'int': None, '<=': None, '}': None, 'T': None, 'Stmt': None, ')': -26, '(': None, '=': None, '/': None, '>=': None, 'S': None, 'IfHeader': None, 'E': None, 'C': None, 'float': None, 'Bnot': None, '$': None, 'Bor': None, 'Stmts': None, '!': None, 'Block': None, '*': None, '&&': -26, '-': 46},
  64:{'{': None, 'else': None, '+': 45, 'WhileExpr': None, 'While': None, 'id': None, 'print': None, '!=': None, 'F': None, 'while': None, '%': None, '==': None, ';': -25, '>': None, 'Expr': None, 'Band': None, 'if': None, '||': -25, 'Else': None, '<': None, 'int': None, '<=': None, '}': None, 'T': None, 'Stmt': None, ')': -25, '(': None, '=': None, '/': None, '>=': None, 'S': None, 'IfHeader': None, 'E': None, 'C': None, 'float': None, 'Bnot': None, '$': None, 'Bor': None, 'Stmts': None, '!': None, 'Block': None, '*': None, '&&': -25, '-': 46},
  65:{'{': None, 'else': None, '+': -29, 'WhileExpr': None, 'While': None, 'id': None, 'print': None, '!=': -29, 'F': None, 'while': None, '%': 51, '==': -29, ';': -29, '>': -29, 'Expr': None, 'Band': None, 'if': None, '||': -29, 'Else': None, '<': -29, 'int': None, '<=': -29, '}': None, 'T': None, 'Stmt': None, ')': -29, '(': None, '=': None, '/': 52, '>=': -29, 'S': None, 'IfHeader': None, 'E': None, 'C': None, 'float': None, 'Bnot': None, '$': None, 'Bor': None, 'Stmts': None, '!': None, 'Block': None, '*': 50, '&&': -29, '-': -29},
  66:{'{': None, 'else': None, '+': -30, 'WhileExpr': None, 'While': None, 'id': None, 'print': None, '!=': -30, 'F': None, 'while': None, '%': 51, '==': -30, ';': -30, '>': -30, 'Expr': None, 'Band': None, 'if': None, '||': -30, 'Else': None, '<': -30, 'int': None, '<=': -30, '}': None, 'T': None, 'Stmt': None, ')': -30, '(': None, '=': None, '/': 52, '>=': -30, 'S': None, 'IfHeader': None, 'E': None, 'C': None, 'float': None, 'Bnot': None, '$': None, 'Bor': None, 'Stmts': None, '!': None, 'Block': None, '*': 50, '&&': -30, '-': -30},
  67:{'{': None, 'else': None, '+': None, 'WhileExpr': None, 'While': None, 'id': None, 'print': None, '!=': None, 'F': None, 'while': None, '%': None, '==': None, ';': -16, '>': None, 'Expr': None, 'Band': None, 'if': None, '||': -16, 'Else': None, '<': None, 'int': None, '<=': None, '}': None, 'T': None, 'Stmt': None, ')': -16, '(': None, '=': None, '/': None, '>=': None, 'S': None, 'IfHeader': None, 'E': None, 'C': None, 'float': None, 'Bnot': None, '$': None, 'Bor': None, 'Stmts': None, '!': None, 'Block': None, '*': None, '&&': 53, '-': None},
  68:{'{': None, 'else': None, '+': -32, 'WhileExpr': None, 'While': None, 'id': None, 'print': None, '!=': -32, 'F': None, 'while': None, '%': -32, '==': -32, ';': -32, '>': -32, 'Expr': None, 'Band': None, 'if': None, '||': -32, 'Else': None, '<': -32, 'int': None, '<=': -32, '}': None, 'T': None, 'Stmt': None, ')': -32, '(': None, '=': None, '/': -32, '>=': -32, 'S': None, 'IfHeader': None, 'E': None, 'C': None, 'float': None, 'Bnot': None, '$': None, 'Bor': None, 'Stmts': None, '!': None, 'Block': None, '*': -32, '&&': -32, '-': -32},
  69:{'{': None, 'else': None, '+': -34, 'WhileExpr': None, 'While': None, 'id': None, 'print': None, '!=': -34, 'F': None, 'while': None, '%': -34, '==': -34, ';': -34, '>': -34, 'Expr': None, 'Band': None, 'if': None, '||': -34, 'Else': None, '<': -34, 'int': None, '<=': -34, '}': None, 'T': None, 'Stmt': None, ')': -34, '(': None, '=': None, '/': -34, '>=': -34, 'S': None, 'IfHeader': None, 'E': None, 'C': None, 'float': None, 'Bnot': None, '$': None, 'Bor': None, 'Stmts': None, '!': None, 'Block': None, '*': -34, '&&': -34, '-': -34},
  70:{'{': None, 'else': None, '+': -33, 'WhileExpr': None, 'While': None, 'id': None, 'print': None, '!=': -33, 'F': None, 'while': None, '%': -33, '==': -33, ';': -33, '>': -33, 'Expr': None, 'Band': None, 'if': None, '||': -33, 'Else': None, '<': -33, 'int': None, '<=': -33, '}': None, 'T': None, 'Stmt': None, ')': -33, '(': None, '=': None, '/': -33, '>=': -33, 'S': None, 'IfHeader': None, 'E': None, 'C': None, 'float': None, 'Bnot': None, '$': None, 'Bor': None, 'Stmts': None, '!': None, 'Block': None, '*': -33, '&&': -33, '-': -33},
  71:{'{': None, 'else': None, '+': None, 'WhileExpr': None, 'While': None, 'id': None, 'print': None, '!=': None, 'F': None, 'while': None, '%': None, '==': None, ';': -18, '>': None, 'Expr': None, 'Band': None, 'if': None, '||': -18, 'Else': None, '<': None, 'int': None, '<=': None, '}': None, 'T': None, 'Stmt': None, ')': -18, '(': None, '=': None, '/': None, '>=': None, 'S': None, 'IfHeader': None, 'E': None, 'C': None, 'float': None, 'Bnot': None, '$': None, 'Bor': None, 'Stmts': None, '!': None, 'Block': None, '*': None, '&&': -18, '-': None},
  72:{'{': None, 'else': None, '+': -40, 'WhileExpr': None, 'While': None, 'id': None, 'print': None, '!=': -40, 'F': None, 'while': None, '%': -40, '==': -40, ';': -40, '>': -40, 'Expr': None, 'Band': None, 'if': None, '||': -40, 'Else': None, '<': -40, 'int': None, '<=': -40, '}': None, 'T': None, 'Stmt': None, ')': -40, '(': None, '=': None, '/': -40, '>=': -40, 'S': None, 'IfHeader': None, 'E': None, 'C': None, 'float': None, 'Bnot': None, '$': None, 'Bor': None, 'Stmts': None, '!': None, 'Block': None, '*': -40, '&&': -40, '-': -40},
  73:{'{': -12, 'else': None, '+': None, 'WhileExpr': None, 'While': None, 'id': None, 'print': None, '!=': None, 'F': None, 'while': -12, '%': None, '==': None, ';': None, '>': None, 'Expr': None, 'Band': None, 'if': -12, '||': None, 'Else': None, '<': None, 'int': None, '<=': None, '}': None, 'T': None, 'Stmt': None, ')': None, '(': None, '=': None, '/': None, '>=': None, 'S': None, 'IfHeader': None, 'E': None, 'C': None, 'float': None, 'Bnot': None, '$': None, 'Bor': None, 'Stmts': None, '!': None, 'Block': None, '*': None, '&&': None, '-': None}
}



def action(lst,table):
    state = 0
    for token in lst:      
        try:
            state = table[state][token[0]]
        except KeyError:
            return err          
        if state == err:        # error state is implicit so fail as soon as you hit an error
            return err
    return state



### Reminder: You should not modify anything above this line unless you check with Professor Snyder first!
<pre>




</pre>

## Introduction

The homework for this week is to extend the ideas from Lab 06 to the while loop, if-then, and if-then-else statements. This involves a consideration of how to modify the flow of control of an Intermediate Code program.
The ideas are similar to the lab, in that we will have to work out how to generate IC statements on the stack of the shift-reduce parser, but we will have to think carefully about the timing of the reduce moves, and in particular how to specify addresses when we do a conditional branch or a jump. 

### The Intermediate Code Language

The language we will use consists of tuples similar to those used in our AST for MyPython, except that there will be no nesting of operations: all instructions will use variables, ints, or floats as operands, and a program will consist of a list of instructions. Hence there will be no tree-structured ASTs, but a flat list of instructions, each of which does one simple operation. 

The instructions of our language are as follows:
<p>
<pre>
<b>IC Statement                         Meaning</b> 

( '+', &lt;dest&gt;, &lt;src1&gt;, &lt;src2&gt; )      &lt;dest&gt; = &lt;src1&gt; + &lt;src2&gt;
( '-', &lt;dest&gt;, &lt;src1&gt;, &lt;src2&gt; )      &lt;dest&gt; = &lt;src1&gt; - &lt;src2&gt;
( '\*', &lt;dest&gt;, &lt;src1&gt;, &lt;src2&gt; )      &lt;dest&gt; = &lt;src1&gt; \* &lt;src2&gt;
( '/', &lt;dest&gt;, &lt;src1&gt;, &lt;src2&gt; )      &lt;dest&gt; = &lt;src1&gt; / &lt;src2&gt;
( '%', &lt;dest&gt;, &lt;src1&gt;, &lt;src2&gt; )      &lt;dest&gt; = &lt;src1&gt; % &lt;src2&gt;
( '==', &lt;dest&gt;, &lt;src1&gt;, &lt;src2&gt; )     &lt;dest&gt; = &lt;src1&gt; == &lt;src2&gt;
( '!=', &lt;dest&gt;, &lt;src1&gt;, &lt;src2&gt; )     &lt;dest&gt; = &lt;src1&gt; != &lt;src2&gt;
( '&lt;', &lt;dest&gt;, &lt;src1&gt;, &lt;src2&gt; )      &lt;dest&gt; = &lt;src1&gt; &lt; &lt;src2&gt;
( '&gt;', &lt;dest&gt;, &lt;src1&gt;, &lt;src2&gt; )      &lt;dest&gt; = &lt;src1&gt; &gt; &lt;src2&gt;
( '&lt;=', &lt;dest&gt;, &lt;src1&gt;, &lt;src2&gt; )     &lt;dest&gt; = &lt;src1&gt; &lt;= &lt;src2&gt;
( '&gt;=', &lt;dest&gt;, &lt;src1&gt;, &lt;src2&gt; )     &lt;dest&gt; = &lt;src1&gt; &gt;= &lt;src2&gt;
( '&&', &lt;dest&gt;, &lt;src1&gt;, &lt;src2&gt; )     &lt;dest&gt; = &lt;src1&gt; && &lt;src2&gt;
( '||', &lt;dest&gt;, &lt;src1&gt;, &lt;src2&gt; )     &lt;dest&gt; = &lt;src1&gt; || &lt;src2&gt;

( 'uminus', &lt;dest&gt;, &lt;src1&gt; )         &lt;dest&gt; = - &lt;src1&gt; 
( '!', &lt;dest&gt;, &lt;src1&gt; )              &lt;dest&gt; = ! &lt;src1&gt; 

( '=', &lt;dest&gt;, &lt;src1&gt; )              &lt;dest&gt; = &lt;src1&gt; 

( 'jump', n )           Unconditional jump: Jump to address n in the IC program
( 'bzero', &lt;src&gt;, m )   Branch-on-0: if &lt;src&gt; == 0, jump to m

( 'print', &lt;str&gt;, &lt;src&gt; )   Print &lt;str&gt; and the value of &lt;src&gt; to the standard output
</pre>
&lt;dest&gt; must be an id tuple and &lt;src&gt;, &lt;src1&gt;, and &lt;src2&gt; can be either id, int, or float tuples;   &lt;str&gt; is a string (a comment); n and m are actual integers (indices into the Code array); 
</p>


### Execution of IC Programs

An IC program is a list of IC statements, which is executed by the following algorithm:


In [131]:
def instruction_to_string(k,instruction):
    if (len(instruction) == 4):
        (op,(_,dest),(_,src1),(_,src2)) = instruction
        return (str(k) + ": " + dest + " = " + str(src1) + " " + op + " " + str(src2))
    elif instruction[0] == 'uminus':
        return (str(k) + ": " + instruction[1][1] + " = - " + str(instruction[2][1]))
    elif instruction[0] == '!':
        return (str(k) + ": " + instruction[1][1] + " = ! " + str(instruction[2][1]))
    elif instruction[0] == '=':
        return (str(k) + ": " + instruction[1][1] + " = " + str(instruction[2][1]))
    elif instruction[0] == 'jump':
        return (str(k) + ": " + "jump " + str(instruction[1]))
    elif instruction[0] == 'bzero':
        return (str(k) + ": " + "bzero " + str(instruction[1][1]) + " " + str(instruction[2]))
    elif instruction[0] == 'print':
        return (str(k) + ": " + "print \"" + str(instruction[1]) + "\" " + str(instruction[2][1]))

def pprint_program(P):
    print("Program:\n")
    for k in range(len(P)):
        print(instruction_to_string(k,P[k]))
        
def lookup(src,Mem):
    if src[0] == 'id':
        return Mem[src[1]]
    elif src[0] == 'int':
        return int(src[1])
    elif src[0] == 'float':
        return float(src[1])
    else:
        print("Unbound source:" + str(src))
        return None

def execute(Code,Verbose=False):

    # memory to hold variable bindings
    Memory = {}
    
    # program counter holding index of next instruction to execute
    PC = 0
    if Verbose:
        print("Execution Trace:")
        print()
        print("Instruction\t\tMemory")
    else:
        print("Execution:\n")
            
    while( PC < len(Code) ):     # when run off end of list, halt program
        if Verbose:
            print("\t\t\t" + str(Memory))
            print(instruction_to_string(PC,Code[PC]))
            
        instruction = Code[PC]
        PC += 1
        # parse instruction
        if len(instruction) == 4:   # binary operation
            (op,(_,name),src1,src2) = instruction
            if op == '+': 
                Memory[name] = lookup(src1,Memory) + lookup(src2,Memory)
            elif op == '-': 
                Memory[name] = lookup(src1,Memory) - lookup(src2,Memory)
            elif op == '*': 
                Memory[name] = lookup(src1,Memory) * lookup(src2,Memory)
            elif op == '/': 
                Memory[name] = lookup(src1,Memory) / lookup(src2,Memory)
            elif op == '%': 
                Memory[name] = lookup(src1,Memory) % lookup(src2,Memory)
            elif op == '<': 
                Memory[name] = int(lookup(src1,Memory) < lookup(src2,Memory))
            elif op == '>': 
                Memory[name] = int(lookup(src1,Memory) > lookup(src2,Memory))
            elif op == '>=': 
                Memory[name] = int(lookup(src1,Memory) >= lookup(src2,Memory))
            elif op == '<=': 
                Memory[name] = int(lookup(src1,Memory) <= lookup(src2,Memory))
            elif op == '==': 
                Memory[name] = int(lookup(src1,Memory) == lookup(src2,Memory))
            elif op == '!=': 
                Memory[name] = int(lookup(src1,Memory) != lookup(src2,Memory))
            elif op == '&&': 
                Memory[name] = int(lookup(src1,Memory) and lookup(src2,Memory))
            elif op == '||': 
                Memory[name] = int(lookup(src1,Memory) or lookup(src2,Memory))
        elif instruction[0] == 'uminus':
            Memory[instruction[1][1]] = - lookup(instruction[2],Memory) 
        elif instruction[0] == '!':
            Memory[instruction[1][1]] = int(not lookup(instruction[2],Memory))
        elif instruction[0] == '=':
            Memory[instruction[1][1]] = lookup(instruction[2],Memory) 
        elif instruction[0] == 'jump':
            PC = instruction[1]
        elif instruction[0] == 'bzero':
            if lookup(instruction[1],Memory) == 0:
                PC = instruction[2]
        elif instruction[0] == 'print':
            print(instruction[1] + str(lookup(instruction[2],Memory)))
        else:
            print("Illegal instruction" + str(instruction))
    
    if Verbose:
            print("\t\t\t" + str(Memory))        
    

### Example 1

Here is the execution of the statement from Lab 06 with x = 10 and 7 = 4 and a print out of the value of x at the end:

In [132]:
Code = [( '=', ('id','x'), ('int',10)),
        ( '=', ('id','y'), ('int',4)),
        ( '*', ('id','_t1'), ('id','x'), ('id','y' )),
        ( '*', ('id','_t2'), ('int',4), ('id','y' )), 
        ( '+', ('id','_t3'), ('id','_t1'), ('id','_t2' )),
        ( '=', ('id','x'), ('id','_t3')),
        ( 'print', "x = ", ('id','x')) ]

print()
pprint_program(Code)
print()
execute(Code)
print()
execute(Code,True)


Program:

0: x = 10
1: y = 4
2: _t1 = x * y
3: _t2 = 4 * y
4: _t3 = _t1 + _t2
5: x = _t3
6: print "x = " x

Execution:

x = 56

Execution Trace:

Instruction		Memory
			{}
0: x = 10
			{'x': 10}
1: y = 4
			{'x': 10, 'y': 4}
2: _t1 = x * y
			{'x': 10, 'y': 4, '_t1': 40}
3: _t2 = 4 * y
			{'x': 10, 'y': 4, '_t1': 40, '_t2': 16}
4: _t3 = _t1 + _t2
			{'x': 10, 'y': 4, '_t1': 40, '_t2': 16, '_t3': 56}
5: x = _t3
			{'x': 56, 'y': 4, '_t1': 40, '_t2': 16, '_t3': 56}
6: print "x = " x
x = 56
			{'x': 56, 'y': 4, '_t1': 40, '_t2': 16, '_t3': 56}


### Example 2: Finding the kth fibonnaci number, for k = 7

In [133]:
Code = [( '=', ('id','fib'), ('int',1)),
        ( '=', ('id','prev_fib'), ('int',0)),
        ( '=', ('id','k'), ('int',7)),
        ( 'bzero', ('id','k'), 9),
        ( '+', ('id','temp'), ('id','fib'), ('id','prev_fib')),
        ( '=', ('id','prev_fib'), ('id','fib')),
        ( '=', ('id','fib'), ('id','temp')),
        ( '-', ('id','k'), ('id','k'), ('int',1)),
        ( 'jump', 3),
        ( 'print', "fib = ", ('id','fib')) ]
     
pprint_program(Code) 
print()
execute(Code)
print()
execute(Code,True)

# Fibonnaci numbers:
# 0 1 2 3 4 5 6  7  8  9
# 1 1 2 3 5 8 13 21 34 55

Program:

0: fib = 1
1: prev_fib = 0
2: k = 7
3: bzero k 9
4: temp = fib + prev_fib
5: prev_fib = fib
6: fib = temp
7: k = k - 1
8: jump 3
9: print "fib = " fib

Execution:

fib = 21

Execution Trace:

Instruction		Memory
			{}
0: fib = 1
			{'fib': 1}
1: prev_fib = 0
			{'fib': 1, 'prev_fib': 0}
2: k = 7
			{'fib': 1, 'prev_fib': 0, 'k': 7}
3: bzero k 9
			{'fib': 1, 'prev_fib': 0, 'k': 7}
4: temp = fib + prev_fib
			{'fib': 1, 'prev_fib': 0, 'k': 7, 'temp': 1}
5: prev_fib = fib
			{'fib': 1, 'prev_fib': 1, 'k': 7, 'temp': 1}
6: fib = temp
			{'fib': 1, 'prev_fib': 1, 'k': 7, 'temp': 1}
7: k = k - 1
			{'fib': 1, 'prev_fib': 1, 'k': 6, 'temp': 1}
8: jump 3
			{'fib': 1, 'prev_fib': 1, 'k': 6, 'temp': 1}
3: bzero k 9
			{'fib': 1, 'prev_fib': 1, 'k': 6, 'temp': 1}
4: temp = fib + prev_fib
			{'fib': 1, 'prev_fib': 1, 'k': 6, 'temp': 2}
5: prev_fib = fib
			{'fib': 1, 'prev_fib': 1, 'k': 6, 'temp': 2}
6: fib = temp
			{'fib': 2, 'prev_fib': 1, 'k': 6, 'temp': 2}
7: k = k - 1
			{'fib': 

## Problem One: Your Turn with IC Programming.... (5 pts)

For this problem, you must write an IC program that calculates the Greatest 
Common Divisor of two integers: the first two lines of your program must assign the values
to the variables x and y, and at the end you must print out the gcd. 

(A) Run your program for x = 12 and y = 28. 

(B) Run your program (i.e., give another instance of the program with 
different values for x and y) for x = 144 and y = 256.

For each, give your program and execute it; you may wish to trace the execution to debug your code, but do not hand in the trace. 

In [134]:
print("Part A:\n")
GCD =  [( '=', ('id','x'), ('int',12)),
        ( '=', ('id','y'), ('int',28)),
        ('%', ('id', '_t1'), ('id', 'y') ,('id', 'x')),
        ( '=', ('id','k'), ('id','_t1')),
        ( 'bzero', ('id','k'), 8),
        ( '=', ('id','x'), ('id', '_t1')),
        ( '=', ('id','y'), ('id', 'x')),
        ( 'jump', 2),
        ( 'print', "GCD = ", ('id','x'))]                   # Your code here
    
pprint_program(GCD) 
print()
execute(GCD)

# Should print out 4

print("\nPart B:\n")
GCD =  [( '=', ('id','x'), ('int',144)),
        ( '=', ('id','y'), ('int',256)),
        ('%', ('id', '_t1'), ('id', 'y') ,('id', 'x')),
        ( '=', ('id','k'), ('id','_t1')),
        ( 'bzero', ('id','k'), 8),
        ( '=', ('id','y'), ('id', 'x')),
        ( '=', ('id','x'), ('id', '_t1')),
        ( 'jump', 2),
        ( 'print', "GCD = ", ('id','x')) ]                   # Your code here
     
pprint_program(GCD) 
print()
execute(GCD)

# Should print out 16

Part A:

Program:

0: x = 12
1: y = 28
2: _t1 = y % x
3: k = _t1
4: bzero k 8
5: x = _t1
6: y = x
7: jump 2
8: print "GCD = " x

Execution:

GCD = 4

Part B:

Program:

0: x = 144
1: y = 256
2: _t1 = y % x
3: k = _t1
4: bzero k 8
5: y = x
6: x = _t1
7: jump 2
8: print "GCD = " x

Execution:

GCD = 16


## Problem Two: Adding Flow-of-Control Statements (20 pts)

In this problem we will add if-then, if-then-else, and while loops to our language; as discussed in lecture, 
we must program this on the stack by carefully considering the timing of the reduce moves,
since our paradigm for "programming the parsing stack" involves adding code to manipulate
information stored on the stack before and after each reduce. 

For expressions and the assignment statement, you need to use the techniques from Lab 06, but separated
into rule 5 (assignment statement) and rules 15 - 40 (arithmetic and boolean expressions). 

For rules 0 - 4, you don't need to do anything, as they do not directly affect the generation of
the IC code. 

For the remaining rules, for flow-of-control instruction, I will post notes from my lecture on Thursday, but here 
is a description of what you need to do on a reduce move for each rule in the grammar:

<pre>
8: Block := While WhileExpr Block 

      As you pop tuples off the stack, save the 
            address of the bzero from the WhileExpr;
            starting address of the while from the While. 

      Generate a jump statement back to starting address of the while;
      backpatch the next address (= the current length of the Code list)
      into the bzero statement. 

9: WhileExpr := ( Expr ) 

      Save the loc (variable) where the Expr is stored, and generate
      a bzero statement with that as src variable, but with no target
      address.

10: While := while 

       Save the address where the while statement starts in the
       While tuple. 

11: Block := IfHeader Block 

        As you pop the stack, save the address of the bzero from the
        IfHeader, and backpatch the next address into the bzero. 

12: IfHeader := if ( Expr ) 

         As you pop the stack, save the location (variable) where the
         value of the Expr is stored, generate a bzero which uses this
         variable as src, but with no target address. 

13: Block := IfHeader Block Else Block 

         As you pop the stack, save the
               address of the jump statement from the Else;
               address of the bzero from the IfHeader;
         back patch the address AFTER the jump into the bzero,
         and the next address into the jump. 

14: Else := else 

       Generate a jump statement with the target address left empty;
       Store the address of the jump statement in the Else tuple.
</pre>

### Backpatching
Backpatching refers to the process of going back to previously-generated IC tuples in 
the code list and filling in the addresses of jumps and bzeros. Unfortunately,
you can't modify tuples in Python once you create them (they are <i> immutable</i>),
so you will have to recreate the tuples. So, if you have previously created
a tuple
<pre>
                   bzero_addr:   ('bzero', src, None)
</pre>
and want to backpatch the branch address <code>next</code> in place of the <code>None</code>, you can't do the first thing you would think of (and which you could do if you were writing the compiler in C or Java:
<pre>
           Code[bzero_addr][2] = next
</pre>
Instead, you have to do something like this:
<pre>
           (_,src,_) = Code[bzero_addr]          # previous tuple
           Code[bzero_addr] = ('bzero',src,next) # reconstruct the new tuple               
</pre>
You would have to do the same for the jump tuple, and in fact, since there is nothing
stored in the jump tuple (unlike the bzero, where you have filled in the source variable), the only reason to create the incomplete jump tuple at first is to "reserve" a place in the code list, and perhaps to help with debugging if you want to look at the code
array before it is done. 

In [135]:
# Code for shift-reduce parser

# pretty-printing parser configurations

def pprint_parser(parsingStack,inputListOfTokens):
    totalWidth = 80
    smallestGap = 6
    largestStackLength = int(totalWidth*0.6 - smallestGap/2)   # most characters to display
    largestInputLength = totalWidth - largestStackLength - smallestGap
    
    p = '| '
    for symbol in parsingStack:
        p = p + tokenToString(symbol) + ' '
    if len(p) > largestStackLength:
        ind = int(len(p) - largestStackLength + 9)
        p = '| ... ' + p[ind:]
        
    q = ""
    for tok in inputListOfTokens[:-1]:
        q = q + tokenToString(tok) + ' '
    if len(inputListOfTokens) > 0:
        q = q + tokenToString(inputListOfTokens[-1]) 

    if len(q) > largestInputLength:
        ind = int(largestInputLength - 9)
        q = q[:ind] + ' ... ' 
    
    q = q + ' |'
        
    p = p + (' ' * (totalWidth - len(p) - len(q)))
    print(p+q)
    
    
# Your code here to manage stack of temporary variables from Lab 06
Global = []

count = 0

def reset():
    global count
    global Global
    count = 0
    Global = []

def getTemp():
    global count
#     print("length" + str(len(Global)))
    if(len(Global) == 0):
        count += 1
        return ('id', '_t' + str(count))
    else:
        return Global.pop()
       

def freeTemp(temp):
    Global.append(temp)

# parse takes a list of tokens and determines if it is a 
# well-formed arithmetic expression.
# This version will also generate a list of IC tuples

def parse(list_of_input_tokens,rules,table, verbose=False):
    
    
    Code = []
    reset()
    # add end of input marker
    list_of_input_tokens.append( (end_of_input,) )
    
    # input stack; use pop(0) to remove front item in list
    input_stack = list_of_input_tokens
    
    # parsing stack; use append to push onto end and pop() to pop from end
    parsing_stack = []
    
    if verbose:
        print('Input Tokens: ' + tokenListToString(input_stack) + '\n')
        pprint_parser(parsing_stack,input_stack) 
    
    while( len(input_stack) > 0 ):
        
        # Now we will "lookahead" at the next symbol on the input and ask the automaton what to do
        # a positive number means to shift, and a negative number -n means to reduce by rule n
        
        n = action(parsing_stack+[input_stack[0]],table)
        if n == accept:   # reduce by start rule, success
            if verbose:
                print("\nAccept!")
            return Code 
        elif n == err:     #  problem on stack!
            if verbose:
                print("\nERROR: No transition here:")
                pprint_parser(parsing_stack,input_stack)
            return None

        elif n > 0:     # shift 
            if verbose:
                print('\naction: ' + str(n) + '\tshift\n')
            parsing_stack.append( input_stack.pop(0)) 
        else:         # reduce by rule -n 
            if verbose:
                print('\naction: ' + str(n) + '\treduce by rule ' + str(-n) + ': ' + rule_to_string(rules[-n])+'\n')
            r = -n
            LHS = rules[r][0]
            
            if r in [0,1,2,3,4,7]:                     # No attributes get passed
                for k in range(len(rules[r][1])):
                    parsing_stack.pop()
                parsing_stack.append( (LHS[0],None) )
            elif r == 6:                               # Stmt := print Expr
                (_,src) = parsing_stack.pop()          # Expr
                parsing_stack.pop()                    # print
                if src[0] == 'id':
                    Code.append( ('print', src[1] + " = ", src) )
                else:
                    Code.append( ('print', "", src) )
                parsing_stack.append( (LHS[0],None) )
            elif r == 5:                               # Stmt := id = Expr
                (_,loc1) = parsing_stack.pop()     # E 
                (loc2,) = parsing_stack.pop()  # =
                loc3 = parsing_stack.pop()         # id 
                Code.append((loc2,loc3,loc1))
                parsing_stack.append( (LHS[0],(loc2,loc3,loc1) ) )
            elif r in [15,17,19,21,28,31,35]:        # Unary rules just pass attributes up
                (_,loc) = parsing_stack.pop()
                parsing_stack.append( (LHS[0],loc) )    
            elif r in [16,18,22,23,24,25,26,27,29,30,32,33,34]:   # same as in Lab 06
                (_,loc1) = parsing_stack.pop()
                (loc2,) = parsing_stack.pop()
                (_,loc3) = parsing_stack.pop()
                if(loc3[0] =='id' and '_' in loc3[1]):
                    freeTemp(loc3[1])
                if(loc1[0] == 'id' and '_' in loc1[1]):
                    freeTemp(loc1[1])
                t1 = getTemp()

                Code.append((loc2,t1,loc3,loc1))
                parsing_stack.append( (LHS[0],t1 ))
            elif r in [37,38,39]:                     # F := id, int, or float
                loc = parsing_stack.pop()
                parsing_stack.append( (LHS[0],loc) )
            elif r in [20]:                           # Bnot := ! Bnot
                (_,loc1) = parsing_stack.pop()      # F  
                (loc2,) = parsing_stack.pop()  # - 
                if(loc1[0] == 'id' and '_' in loc1[1]):
                    freeTemp(loc1)
                t2 = getTemp()

                Code.append(('!',t2,loc1))
                parsing_stack.append( (LHS[0],t2) )
            elif r in [36]:                           # F := - F  
                (_,loc1) = parsing_stack.pop()      # F  
                (loc2,) = parsing_stack.pop()  # - 
                if(loc1[0] == 'id' and '_' in loc1[1]):
                    freeTemp(loc1)
                t2 = getTemp()

                Code.append(('uminus',t2,loc1))
                parsing_stack.append( (LHS[0],t2) )
            elif r == 40:                             # F := ( Expr )
                parsing_stack.pop()               # (
                (_,loc2) = parsing_stack.pop()      # E
                parsing_stack.pop()                # )
                parsing_stack.append( (LHS[0],loc2) )
            # if statement
            elif r == 11:                             # Block := IfHeader Block
                parsing_stack.pop()                   # Block
                (_,addr) = parsing_stack.pop()                   # IfHeader 
                (_,src,_) = Code[addr]
                Code[addr] = ('bzero',src,len(Code))
                parsing_stack.append( (LHS[0],None) ) 
            elif r == 12:                             # IfHeader := if ( Expr )
                parsing_stack.pop()                   # )
                (_,loc) = parsing_stack.pop()                   # Expr
                parsing_stack.pop()                   # (
                parsing_stack.pop()                   # if
                Code.append(('bzero',loc,None))
                parsing_stack.append( (LHS[0], (len(Code)-1)) ) # store addr of bzero in IfHeader
            # if-then-else statement
            elif r == 13:                             # Block := IfHeader Block Else Block
                parsing_stack.pop()                   # Block
                (_,jaddr) = parsing_stack.pop()                   # Else 
                (_,addr) = parsing_stack.pop()                   # Block
                (_,baddr) = parsing_stack.pop()                   # IfHeader 
                (_,src,_) = Code[baddr]
                Code[baddr] = ('bzero',src,jaddr+1)
#                 (_,src) = Code[jaddr]
                Code[jaddr] = ('jump',len(Code))
                parsing_stack.append( (LHS[0],None) ) 
            elif r == 14:                             # Else := else
                parsing_stack.pop()                   # else 
                Code.append(('jump',None))
                parsing_stack.append( (LHS[0],len(Code)-1) ) # store address in Then         
            # while loop
            elif r == 10:                             # While := while
                parsing_stack.pop()                   # while
                parsing_stack.append( (LHS[0],len(Code)) ) # store start of while in While
            elif r == 9:                              # WhileExpr := ( Expr )
                parsing_stack.pop()                   # )
                (_,loc) = parsing_stack.pop()                   #  Expr
                Code.append(('bzero',loc,None))
                parsing_stack.pop()                   # (  
                parsing_stack.append( (LHS[0],(len(Code)-1)) ) # store addr of bzero in WhileExpr
            elif r == 8:                              # Block := While WhileExpr Block
                parsing_stack.pop()                   # Block
                (_,baddr) = parsing_stack.pop()                   # WhileExpr
                (_,src,_) = Code[baddr]
                Code[baddr] = ('bzero',src,len(Code)+1)
                (_,loc) = parsing_stack.pop()         # While
                Code.append(('jump',loc))
                parsing_stack.append( (LHS[0],None) ) 
            else:
                print("parse stack error: action " + str(n))
                return None
   
        if verbose:
            pprint_parser(parsing_stack,input_stack)
            
    return Code     # this will never be executed



In [136]:
def test(P,verbose=False):
    if(P == None):
        print("Syntax error in program!")
        for i in P:
            print(i)
    else:
        pprint_program(P)
        print()
        execute(P,verbose)

In [137]:
# Test 1: make sure expressions still work, as in lab 06

s = '''

x = 6; 
y = 8;
z = x * y / 3 + -x; 
w = z % (x - 2);
print w;

'''

P = parse(lexer(s),Grammar_HW06,table)

test(P) 
#test(P,True)           # if you need to debug


Program:

0: x = 6
1: y = 8
2: _t1 = x * y
3: _t1 = _t1 / 3
4: _t2 = - x
5: _t2 = _t1 + _t2
6: z = _t2
7: _t1 = x - 2
8: _t1 = z % _t1
9: w = _t1
10: print "w = " w

Execution:

w = 2.0


In [138]:
# Test 2: make sure boolean expressions work

s = '''

x = 1; 
y = 0;
z = x && y || 0;
w = !  z;
print w;

'''

P = parse(lexer(s),Grammar_HW06,table)

test(P) 
#test(P,True)           # if you need to debug

Program:

0: x = 1
1: y = 0
2: _t1 = x && y
3: _t1 = _t1 || 0
4: z = _t1
5: _t2 = ! z
6: w = _t2
7: print "w = " w

Execution:

w = 1


In [139]:
# Test 3: if statements

s = '''

x = 1; 
y = 0;
if(x && y || 1) {
    y = 4;
}
print y;

x = 1; 
y = 0;
if(0) {
    y = 4;
}
print y;

'''

P = parse(lexer(s),Grammar_HW06,table)

test(P) 
#test(P,True)           # if you need to debug

Program:

0: x = 1
1: y = 0
2: _t1 = x && y
3: _t1 = _t1 || 1
4: bzero _t1 6
5: y = 4
6: print "y = " y
7: x = 1
8: y = 0
9: bzero 0 11
10: y = 4
11: print "y = " y

Execution:

y = 4
y = 0


In [140]:
# Test 4: if statements

s = '''

x = 1; 
y = 0;
z = 2;
if(! (x && y)) {
    y = 4;
    if(x) {
        print x;
    }
    print y;       
}
print z;

'''

P = parse(lexer(s),Grammar_HW06,table)

test(P) 
#test(P,True)           # if you need to debug

Program:

0: x = 1
1: y = 0
2: z = 2
3: _t1 = x && y
4: _t1 = ! _t1
5: bzero _t1 10
6: y = 4
7: bzero x 9
8: print "x = " x
9: print "y = " y
10: print "z = " z

Execution:

x = 1
y = 4
z = 2


In [141]:
# Test 5: if-then-else statements

s = '''

x = 1; 
if( x ) {
    y = 4;     
}
else {
    y = 10;
}
print y;

x = 0; 
if( x ) {
    y = 4;     
}
else {
    y = 10;
}
print y;

'''

P = parse(lexer(s),Grammar_HW06,table)

test(P) 
#test(P,True)           # if you need to debug

Program:

0: x = 1
1: bzero x 4
2: y = 4
3: jump 5
4: y = 10
5: print "y = " y
6: x = 0
7: bzero x 10
8: y = 4
9: jump 11
10: y = 10
11: print "y = " y

Execution:

y = 4
y = 10


In [142]:
# Test 6: if-then-else statements  Try various values of x!

s = '''

x = 2;
if( x == 0 ) {
    y = 0;     
}
else { if ( x == 1 ) {
    y = 1;
} else { if ( x == 2) {
    y = 2;
} else { 
    y = 3;
} } }

print x;
print y;

'''

P = parse(lexer(s),Grammar_HW06,table)

test(P) 
#test(P,True)           # if you need to debug

Program:

0: x = 2
1: _t1 = x == 0
2: bzero _t1 5
3: y = 0
4: jump 14
5: _t2 = x == 1
6: bzero _t2 9
7: y = 1
8: jump 14
9: _t3 = x == 2
10: bzero _t3 13
11: y = 2
12: jump 14
13: y = 3
14: print "x = " x
15: print "y = " y

Execution:

x = 2
y = 2


In [143]:
# Test 7: while loops -- sum the numbers from 1 to 10

s = '''

k = 1;
sum = 0;

while ( k <= 10 ) {
    sum = sum + k;
    k = k + 1;
}

print sum;

'''

P = parse(lexer(s),Grammar_HW06,table)

test(P) 
#test(P,True)           # if you need to debug

Program:

0: k = 1
1: sum = 0
2: _t1 = k <= 10
3: bzero _t1 9
4: _t2 = sum + k
5: sum = _t2
6: _t3 = k + 1
7: k = _t3
8: jump 2
9: print "sum = " sum

Execution:

sum = 55


In [144]:
# Test 8: nested while loops -- for (n,m) with 1 <= n <= 3 and 1 <= m <= 4,
# count for how many pairs n evenly divides m.

s = '''

n = 1;

count = 0;

while ( n <= 3 ) {
    m = 1;
    while ( m <= 4 ) {
        if ( m % n == 0 ) {
           count = count + 1;
        }
        m = m + 1;
    }
    n = n + 1;
}

print count;

'''

P = parse(lexer(s),Grammar_HW06,table)

test(P) 
#test(P,True)           # if you need to debug

Program:

0: n = 1
1: count = 0
2: _t1 = n <= 3
3: bzero _t1 18
4: m = 1
5: _t2 = m <= 4
6: bzero _t2 15
7: _t3 = m % n
8: _t3 = _t3 == 0
9: bzero _t3 12
10: _t4 = count + 1
11: count = _t4
12: _t5 = m + 1
13: m = _t5
14: jump 5
15: _t6 = n + 1
16: n = _t6
17: jump 2
18: print "count = " count

Execution:

count = 7


## Problem Three:  Your Turn Again!  (10 pts)

For this problem, I would like you to write three programs in this language. You can
see the exact syntax in the grammar, but these are main things to keep in mind:
    
  - All statements must end with a semicolon;
  - Blocks { ... } do NOT end in a semicolon;
  - if, if-then-else, and while loops MUST use blocks, not single statements;
  - There are no break or continue statements.  
 
(A) Write a program to determine the value of &pi; using the first 1,000,000 terms of
the series:
    
    π/4 = 1 − 1/3 + 1/5 − 1/7 + 1/9 − 1/11 + 1/13 − 1/15 + 1/17 − .....

(B) Write a program which uses the Newton-Raphson method to find the square root of 3 
to an accuracy of 0.0001. 

(C) Write a program to print out the first 100 prime numbers.

In [145]:
#Part A
s = '''

k = 0;
total = 0;
decimal = 1;
while ( k <= 1000000 ) {
    if(decimal % 4 == 3){
        total = total - 1/decimal;
        k = k + 1;
        decimal = decimal + 2;
    }
    else{
        total = total + 1/decimal;
        k = k + 1;
        decimal = decimal + 2; 
    }
}
total = 4 * total;
print total;

'''

P = parse(lexer(s),Grammar_HW06,table)

test(P) 
#test(P,True)           # if you need to debug

Program:

0: k = 0
1: total = 0
2: decimal = 1
3: _t1 = k <= 1000000
4: bzero _t1 24
5: _t2 = decimal % 4
6: _t2 = _t2 == 3
7: bzero _t2 16
8: _t3 = 1 / decimal
9: _t3 = total - _t3
10: total = _t3
11: _t4 = k + 1
12: k = _t4
13: _t5 = decimal + 2
14: decimal = _t5
15: jump 23
16: _t6 = 1 / decimal
17: _t6 = total + _t6
18: total = _t6
19: _t7 = k + 1
20: k = _t7
21: _t8 = decimal + 2
22: decimal = _t8
23: jump 3
24: _t9 = 4 * total
25: total = _t9
26: print "total = " total

Execution:

total = 3.1415936535887745


In [146]:
#Part B

s = '''

num = 3;
x1 = (num * 1)/2;
x2 = (x1 + ( num / x1)) / 2;
while ( (x2 - x1) >= 0.0001) {
    x1 = x2;
    x2 = (x1 + (num / x1)) / 2;
}

print x2;

'''

P = parse(lexer(s),Grammar_HW06,table)

test(P) 
#test(P,True)           # if you need to debug

Program:

0: num = 3
1: _t1 = num * 1
2: _t1 = _t1 / 2
3: x1 = _t1
4: _t2 = num / x1
5: _t2 = x1 + _t2
6: _t2 = _t2 / 2
7: x2 = _t2
8: _t3 = x2 - x1
9: _t3 = _t3 >= 0.0001
10: bzero _t3 17
11: x1 = x2
12: _t4 = num / x1
13: _t4 = x1 + _t4
14: _t4 = _t4 / 2
15: x2 = _t4
16: jump 8
17: print "x2 = " x2

Execution:

x2 = 1.7321428571428572


In [147]:
#Part C

s = '''

i = 1;
num = 0;

while ( i <= 100 ) {
    counter = 0;
    num = i;
    while(num >=1){
       if( i % num == 0){
           counter = counter + 1;
       }
       num = num - 1;
    }
    if( counter == 2){
        print i;
    }
    i = i + 1;
}


'''

P = parse(lexer(s),Grammar_HW06,table)

test(P) 
#test(P,True)           # if you need to debug


Program:

0: i = 1
1: num = 0
2: _t1 = i <= 100
3: bzero _t1 22
4: counter = 0
5: num = i
6: _t2 = num >= 1
7: bzero _t2 16
8: _t3 = i % num
9: _t3 = _t3 == 0
10: bzero _t3 13
11: _t4 = counter + 1
12: counter = _t4
13: _t5 = num - 1
14: num = _t5
15: jump 6
16: _t6 = counter == 2
17: bzero _t6 19
18: print "i = " i
19: _t7 = i + 1
20: i = _t7
21: jump 2

Execution:

i = 2
i = 3
i = 5
i = 7
i = 11
i = 13
i = 17
i = 19
i = 23
i = 29
i = 31
i = 37
i = 41
i = 43
i = 47
i = 53
i = 59
i = 61
i = 67
i = 71
i = 73
i = 79
i = 83
i = 89
i = 97
